# **Table of Contents**
1. Introduction
2. Pipeline Input Parameters
3. Preprocessing
4. Training
5. Evaluation
6. Register Model
7. Accuracy Condition Step
8. Pipeline Creation
9. Make New Model From the Already Existing Pipeline

# **I. Introduction**
In this stage, we will initialize some important variables. Most of these variables are variables that needed to be initialized in order for SageMaker to be able to run. Beside of that, there are some variables that are best to be initialized at the front so it will make the whole work structurally better. 

First, we need to ensure that the latest version of SageMaker is installed. 

## Ensure that newest sagemaker is installed

In [ ]:
!pip install -U sagemaker --quiet # Ensure latest version of SageMaker is installed

## Import Libraries & the Tools

In [ ]:
import sagemaker

session = sagemaker.session.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()
bucket = session.default_bucket()

## Put Important Names

In [ ]:
model_package_group_name = "Hate-Speech-Classifier"  # Model name in model registry
prefix = "sagemaker/HS-Classifier"  # Prefix to S3 artifacts
pipeline_name = "HSPipeline"  # SageMaker Pipeline name

## Upload Data

In [ ]:
input_data_uri = session.upload_data(path="dataset/hate-speech-dataset.csv",
                                     bucket=bucket
                                     key_prefix=prefix + "/data")

# **II. Pipeline Input Parameters**
Below are parameters that are initialized for pipeline. Later on, if we want to make a new model with newer dataset and new parameterss, we just need to run them in our pipeline code. 

In [ ]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.large")
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
input_data = ParameterString(name="InputData", default_value=input_data_uri)
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")

epochs = ParameterInteger(name='Epochs', default_value=3)
train_batch_size = ParameterInteger(name='TrainBatchSize', default_value=32)
model_name = ParameterString(name='ModelName', default_value='bert-base-cased')
training_instance_type = ParameterString(name='TrainingInstanceType', default_value='ml.p3.2xlarge')
training_instance_count = ParameterInteger(name='TrainingInstanceCount', default_value=1)

# **III. Preprocessing**
In this step, we preprocess the input dataset so it will be more suitable to be trained. We are doing this by using Scikit-Learn Processor, a preprocessing tool based on Scikit-Learn that are built within SageMaker. In order for this processor to work, we need to prepare a preprocessing script. In this step, we will divide the data into two groups, _train dataset_ and _validation dataset_. Train dataset will be used for training the model and validation dataset will be used for validating the quality of the model.  

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables

# Create SKlearn processor object
# The object contains information about what instance type to use, the IAM role to use etc.
# A managed processor comes with a preconfigured container, so only specifying version is required.
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="processing-job",
)

# Use the sklearn_processor in a Sagemaker pipelines ProcessingStep
step_preprocess_data = ProcessingStep(
    name="Preprocess-Data",
    processor=sklearn_processor,
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    
    outputs=[ProcessingOutput(output_name="train", source="/opt/ml/processing/train",
                              destination=Join(on="/", values=["s3://{}".format(bucket),
                                                               prefix, 
                                                               ExecutionVariables.PIPELINE_EXECUTION_ID,
                                                               "train"])),
             
             ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",
                              destination=Join(on="/", values=["s3://{}".format(bucket),
                                                               prefix,
                                                               ExecutionVariables.PIPELINE_EXECUTION_ID,
                                                               "validation"]))
            ],
    code="scripts/preprocess.py",
)

# **IV. Training**
This is the Training Step, a step where we will use the _train dataset_ that we obtained from the previous step to train our model. We will use **Bert-Base-Cased** architecture for our model. Just like the previous step, we also need to prepare a python script to execute this step. We will use HuggingFace function that is built within SageMaker. There are some parameters that are included in our pipeline parameters. By doing this, we can change their values in the future to experiment then we can get the best model possible. 

In [ ]:
from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': epochs,
                 'train_batch_size': train_batch_size,
                 'model_name':model_name
                 }
huggingface_estimator = HuggingFace(entry_point='scripts/train.py',
                                    instance_type=training_instance_type,
                                    instance_count=training_instance_count,
                                    role=role,
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',
                                    hyperparameters = hyperparameters)

step_train_model = TrainingStep(
    name="Train-Model",
    estimator=huggingface_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_preprocess_data.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

# **V. Evaluation**
In this step, we will evaluate the quality of the model that we already obtained from the previous step. We will evaluate this model using _validation dataset_ that we obtained from preprocessing step. We will use Scikit-Learn Processor, the same type of processor that we used for preprocessing. Similar to two previous steps, we also need to prepare a python script to execute this step. The final product of this step is **Evaluation Report** that has the full report of our model's quality including its accuracy, precision, recall, and confusion matrix. 

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

evaluate_model_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={"AWS_DEFAULT_REGION": region},
    max_runtime_in_seconds=7200,
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

# Use the evaluate_model_processor in a Sagemaker pipelines ProcessingStep.
step_evaluate_model = ProcessingStep(
    name="Evaluate-Model",
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_preprocess_data.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,  
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "evaluation-report",
                ],
            ),
        ),
    ],
    code="scripts/evaluate.py",
    property_files=[evaluation_report],
)

# **VI. Register Model**
If the trained model meets the model performance requirements a new model version is registered with the model registry for further analysis. To attach model metrics to the model version, create a **Model Metrics** object using the evaluation report created in the evaluation step. Then, create the **Register Model** step.

In [ ]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

# Create ModelMetrics object using the evaluation report from the evaluation step
# A ModelMetrics object contains metrics captured from a model.
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                step_evaluate_model.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
                    "S3Uri"
                ],
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    )
)

# Crete a RegisterModel step, which registers the model with Sagemaker Model Registry.
step_register_model = RegisterModel(
    name="Register-Model",
    estimator=huggingface_estimator,
    model_data=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

# **VII. Accuracy Condition Step**
Adding conditions to the pipeline is done with a **Condition Step**. In this case, we only want to register the new model version with the model registry if the new model meets an accuracy condition. We can set the acccuracy condition based on our need. 

In [ ]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# Create accuracy condition to ensure the model meets performance requirements.
# Models with a test accuracy lower than the condition will not be registered with the model registry.
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluate_model.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.accuracy.value",
    ),
    right=0.7,
)

# Create a Sagemaker Pipelines ConditionStep, using the condition above.
# Enter the steps to perform if the condition returns True / False.
step_cond = ConditionStep(
    name="Accuracy-Condition",
    conditions=[cond_gte],
    if_steps=[step_register_model],
    else_steps=[],
)

# **VIII. Pipeline Creation**
Now with all steps exist, we can combine them all to make a pipeline. 

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

# Create a Sagemaker Pipeline.
# Each parameter for the pipeline must be set as a parameter explicitly when the pipeline is created.
# Also pass in each of the steps created above.
# Note that the order of execution is determined from each step's dependencies on other steps,
# not on the order they are passed in below.
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        model_approval_status,
        input_data,
    ],
    steps=[step_preprocess_data, step_train_model, step_evaluate_model, step_cond],
)

# Submit pipline
pipeline.upsert(role_arn=role)

# Execute pipeline using the default parameters.
execution = pipeline.start()

execution.wait()

# List the execution steps to check out the status and artifacts:
execution.list_steps()

# **IX. Make New Model From the Already Existing Pipeline**
We can run this model if we have new dataset. When we know that we need new model to match the newest dataset, we can obtain a new model only by running below code because we already have a pipeline. 

In [ ]:
new_input_data_uri = session.upload_data(path="dataset/new-hate-speech-dataset.csv",
                                         bucket=bucket,
                                         key_prefix=prefix + "/new-data")



# Execute pipeline with explicit parameters
execution = pipeline.start(
    parameters=dict(
        InputData=new_input_data_uri,
    )
)

execution.wait()